<div style="text-align: center;">
  <img src="https://github.com/Hack-io-Data/Imagenes/blob/main/01-LogosHackio/logo_naranja@4x.png?raw=true" alt="esquema" />
</div>

# Laboratorio Limpieza de Datos

En este laboratorio usaremos el DataFrame de Netflix completo creado en los primeros laboratorios de Pandas. 

**Instrucciones:**

1. Lee cuidadosamente el enunciado de cada ejercicio.

2. Implementa la solución en la celda de código proporcionada.

3. Documenta todas las funciones creadas durante el ejercicio. 

4. Debes incluir después de cada gráfica la interpretación de las mismas en una celda de markdown. 

In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None) 

In [2]:
df = pd.read_csv("./df_resultante.csv", index_col=0)
df.head(1)

,show_id,type,Title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,Genre,Premiere,Runtime,IMDB Score,Language,_merge
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,2021-09-25,2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm...",Documentary,2020-10-02,90.0,7.5,English,both


## Parte 1: Limpieza y Preparación de Datos

#### Ejercicio 1: Estandarización y limpieza de columnas

En este ejercicio, debes limpiar y estandarizar algunas columnas clave para hacerlas más manejables y consistentes en tus análisis. Específicamente, trabajarás con las columnas `date_added` y `duration` para convertirlas a un formato uniforme y estructurado.

Instrucciones:

1. **Convertir la columna `date_added`**: La columna `date_added` contiene fechas en formato de texto. Debes convertirla a un formato `datetime` que pandas pueda entender y manejar fácilmente.

2. **Limpiar la columna `duration`**: La columna `duration` tiene valores en diferentes formatos como "1 Season", "2 Seasons", "90 min", etc. Tu tarea es extraer el número (ya sea el número de temporadas o la cantidad de minutos) y crear una nueva columna llamada `duration_cleaned` con esos valores estandarizados.


**Resultado Esperado:**
Deberás obtener algo como esto:

| duration   | duration_cleaned |
|------------|-----------------|
| 1 Season   | 1               |
| 90 min     | 90              |
| 2 Seasons  | 2               |
| 45 min     | 45              |
| 3 Seasons  | 3               |

In [ ]:
df["date_added"].head(1)
# Aprovechando el formato año-mes-guion, mantendremos ese formato pero en tipo fecha

0    2021-09-25
Name: date_added, dtype: object

In [7]:
df['date_added'] = pd.to_datetime(df['date_added'], format='%Y-%m-%d')

In [ ]:
df['date_added'].head(1)
# Podemos ver como dtype nos indica que es de tipo fecha

0   2021-09-25
Name: date_added, dtype: datetime64[ns]

#### Ejercicio 2: Normalización de la columna `rating`

La columna `rating` tiene diferentes calificaciones como `PG`, `PG-13`, `R`, entre otras. Debes categorizar estas calificaciones en tres grupos:

- **'General Audience'** para calificaciones como `G`, `PG`.

- **'Teens'** para calificaciones como `PG-13`, `TV-14`.

- **'Adults'** para calificaciones como `R`, `TV-MA`.


In [13]:
print("Estos son los diferentes valores que encontramos en la columna rating", df["rating"].unique())

Estos son los diferentes valores que encontramos en la columna rating ['PG-13' 'TV-MA' 'PG' 'TV-14' 'TV-PG' 'TV-Y' 'TV-Y7' 'R' 'TV-G' 'G'
 'NC-17' '74 min' '84 min' '66 min' 'NR' nan 'TV-Y7-FV' 'UR']


In [14]:
def normalizar_rating(rating):
    rating = str(rating).upper().strip()
    
    general_audience = {'G', 'PG'}
    teens = {'PG-13', 'TV-14'}
    adults = {'R', 'TV-MA'}
    
    if rating in general_audience:
        return 'General Audience'
    elif rating in teens:
        return 'Teens'
    elif rating in adults:
        return 'Adults'
    else:
        return 'Not classified'

df['rating_normalizado'] = df['rating'].apply(normalizar_rating)

In [18]:
print("Así quedaría la nueva columna después de normalizar rating", df["rating_normalizado"].value_counts())

Así quedaría la nueva columna después de normalizar rating rating_normalizado
Adults              4006
Teens               2650
Not classified      1823
General Audience     328
Name: count, dtype: int64


#### Ejercicio 3: Creación de una columna personalizada basada en el elenco

Vamos a identificar si un actor clave como `Leonardo DiCaprio`, `Tom Hanks`, o `Morgan Freeman` aparece en el elenco.

Usa `apply` y una función lambda para crear una nueva columna llamada `has_famous_actor` que contenga `True` si alguno de estos actores está en la lista de `cast` y `False` en caso contrario.

In [ ]:
actores_famosos = ["Leonardo DiCaprio", "Tom Hanks", "Morgan Freeman"]

df['has_famoues_actor'] = df['cast'].apply(
    lambda x: True if any(
        actor.strip() in actores_famosos 
        for actor in str(x).split(',')
    ) else False
)

# La columna cast, al ser una lista en forma de String, separando los acotres por 
# comas, requiere hacer el split() para separarlos y poder hacer la comparación

In [28]:
print("Contamos con 35 películas donde sí se da esta condición", df["has_famoues_actor"].value_counts())

Contamos con 35 películas donde sí se da esta condición has_famoues_actor
False    8772
True       35
Name: count, dtype: int64


#### Ejercicio 4: Creación de una columna personalizada usando lógica condicional

Vamos a crear una columna llamada `is_recent` que identifique si un título fue lanzado en los últimos 5 años.

Crea una función para marcar con `True` si el título es reciente (lanzado en los últimos 5 años) y `False` si no lo es.

In [30]:
from datetime import datetime

def is_recent(release_year):
        current_year = datetime.now().year
        año_pelicula = pd.to_numeric(release_year)
        if current_year - año_pelicula <= 5:
            return True
        else:
            return False

df['is_recent'] = df['release_year'].apply(is_recent)

In [34]:
print("Distribución de las películas por is_recent", df["is_recent"].value_counts())

Distribución de las películas por is_recent is_recent
False    6232
True     2575
Name: count, dtype: int64


#### Ejercicio 5: Clasificación de películas por década

En este ejercicio, tu objetivo es categorizar los años de lanzamiento de las películas o series en décadas. La columna `release_year` contiene el año de lanzamiento y debes crear una nueva columna llamada `decade` que indique la década correspondiente, como "1990s", "2000s", etc.


In [35]:
def get_decade(release_year):
    año_menos_ultimo_digito = pd.to_numeric(release_year)
    return (año_menos_ultimo_digito // 10) * 10
# Usando el símbolo de la doble división, no nos devuelve 
# decimales, así quitamos el último dígito y luego añadimos un 0 para indicar la década
df['decade'] = df['release_year'].apply(get_decade)

In [ ]:
print("Distribución de las películas décadas", df["decade"].value_counts())

Distribución de las películas por is_recent decade
2010    5927
2020    1545
2000     810
1990     274
1980     129
1970      70
1960      25
1940      15
1950      11
1920       1
Name: count, dtype: int64


#### Ejercicio 6: Extracción de información

Para practicar la extracción de información:

1. **Extrae el primer actor** de la lista en la columna `cast` y crea una nueva columna llamada `first_actor`.

2. **Extrae el primer nombre del director** y guárdalo en una columna llamada `first_name_director`.


In [43]:
def get_first_actor(names_string):
    if pd.isna(names_string) or names_string == '':
        return None
    return names_string.split(',')[0].strip()

def get_first_director(names_string):
    if pd.isna(names_string) or names_string == '':
        return None
    first_name = names_string.split(',')[0].strip().split(' ')[0]
    return first_name

df['first_actor'] = df['cast'].apply(get_first_actor)
df['first_name_director'] = df['director'].apply(get_first_director)

In [47]:
df.head(1)

,show_id,type,Title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,Genre,Premiere,Runtime,IMDB Score,Language,_merge,rating_normalizado,has_famoues_actor,is_recent,decade,first_actor,first_name_director
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,2021-09-25,2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm...",Documentary,2020-10-02,90.0,7.5,English,both,Teens,False,True,2020,None,Kirsten


#### Ejercicio 7: Limpieza de la columna `cast`

La columna `cast` contiene una lista de actores separados por comas. Tu objetivo es realizar las siguientes tareas:

1. **Reemplaza los valores nulos** en la columna `cast` por "sin información".

2. **Contar el número de actores** en cada entrada y crear una nueva columna llamada `num_cast`.

3. **Normalizar los nombres**: Asegúrate de que los nombres de los actores estén en un formato consistente (por ejemplo, quitar espacios adicionales).


In [52]:
df['cast'] = df['cast'].fillna('sin información')

In [56]:
df.head(1)

,show_id,type,Title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,Genre,Premiere,Runtime,IMDB Score,Language,_merge,rating_normalizado,has_famoues_actor,is_recent,decade,first_actor,first_name_director,recurrent_director
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,sin información,United States,2021-09-25,2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm...",Documentary,2020-10-02,90.0,7.5,English,both,Teens,False,True,2020,None,Kirsten,No


In [60]:
def num_cast(cast):
    if cast == 'sin información':
        return 0
    return len(cast.split(','))

df['num_cast'] = df['cast'].apply(num_cast)


#### Ejercicio 9: Identificación de Directores Recurrentes

En este ejercicio, debes identificar los directores que aparecen más de una vez en el conjunto de datos. Realiza los siguientes pasos:

1. **Reemplaza los valores nulos** en la columna `director` por "sin información".

3. **Cuenta cuántas veces aparece cada director** en la columna creada en el ejercicio 6.

4. **Filtra aquellos directores que aparecen más de una vez** y crea una nueva columna llamada `recurrent_director` donde se indique "Yes" si el director aparece varias veces o "No" en caso contrario.

In [ ]:
df['director'] = df['director'].fillna('no information')

director_counts = df['director'].value_counts()

def is_recurring_director(director):
    return 'Yes' if director_counts[director] > 1 else 'No'

df['recurrent_director'] = df['director'].apply(is_recurring_director)

In [51]:
print("Distribución de directores que aparecen más de 1 vez", df["recurrent_director"].value_counts())

Distribución de directores que aparecen más de 1 vez recurrent_director
Yes    5146
No     3661
Name: count, dtype: int64
